In [1]:
from src.features.mapping import SemanticMap3DBuilder
import numpy as np
import pathlib
import os
from yacs.config import CfgNode
from src.model.perception.perception_pipeline_config import get_perception_cfg
from src.model.perception.evaluation import get_ground_truth
from src.model.perception.labeler import LabelGenerator
from matplotlib import pyplot as plt
import torch
from src.model.perception.model_wrapper import ModelWrapper
from PIL import Image
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [2]:
if pathlib.PurePath(os.getcwd()).name == "notebooks":
    print(pathlib.PurePath(os.getcwd()).name)
    os.chdir("..")

notebooks


In [5]:
scene_id = '00800-TEEsavR23oF'
epoch = 0
TRAJECTORY_DIR = pathlib.Path('data') / 'interim' / 'trajectories' / '23-06-11,16:09:05' / f'epoch_{epoch}' / scene_id
SEMANTIC_INFO_PATH = (pathlib.Path('data') / 'raw' / 'minival' / 'scene_datasets' / 'hm3d' / 'minival' 
                      / scene_id / f'{scene_id.split("-")[1]}.semantic.txt')

In [18]:
semantics_list = []
rgb_list = []
truth_dicts = []
for i in [142, 143, 144]:
    semantics = np.load(TRAJECTORY_DIR / 'Semantic' / f'{i}.npy')
    semantics_list.append(semantics)
    rgb_list.append(Image.open(TRAJECTORY_DIR / 'RGB' / f'{i}.png'))
    truth_dicts.append(get_ground_truth(semantics, SEMANTIC_INFO_PATH))

In [24]:
from src.model.perception.perception_pipeline_config import get_perception_cfg
from src.data.MaskRCNNEvaluationDataset import MaskRCNNEvaluationDataset, eval_collate_fn
from pathlib import Path
from torch.utils.data import DataLoader


perception_cfg = get_perception_cfg()
perception_cfg.DATA_PATHS.TRAJECTORIES_DIR = str(Path(
        perception_cfg.DATA_PATHS.INTERIM_DATA_DIR, "trajectories", "23-06-11,16:09:05"))

eval_dataset = MaskRCNNEvaluationDataset(perception_cfg.DATA_PATHS, perception_cfg.DATA_GENERATOR.SPLIT, 0)
eval_dataloader = DataLoader(eval_dataset,
                                batch_size=perception_cfg.TRAINING.BATCH_SIZE,
                                shuffle=perception_cfg.TRAINING.SHUFFLE,
                                num_workers=perception_cfg.TRAINING.NUM_WORKERS,
                                collate_fn=eval_collate_fn)



In [35]:
for images, semantics, semantic_info_paths in eval_dataloader:
    ground_truths = [get_ground_truth(semantics[i], semantic_info_paths[i]) for i in range(len(images))]
    print(model.get_metrics(images, ground_truths))

0.35634133219718933
0.2227722704410553
0.11303630471229553
0.0
0.08580858260393143
0.2940044105052948
0.30845391750335693
0.0717821791768074
0.11689918488264084
0.1158415824174881
0.38151815533638
0.07897218316793442
0.06435643881559372
0.0
0.08415841311216354
0.3014301359653473
0.5940043926239014
0.0
0.22442243993282318
0.0
0.2564827799797058
0.17326731979846954
0.0631188154220581
0.30693069100379944
0.012376237660646439
0.15951594710350037
0.1262376308441162
0.09460946172475815
0.1920173168182373
-1.0
0.2524752616882324
0.11221121996641159
0.43399345874786377
0.1283828467130661
0.021039603278040886
0.09075907617807388
0.08415841311216354
0.18069307506084442
0.08085808902978897
-1.0
0.422442227602005
0.5049505233764648
0.3333333432674408
-1.0
0.24917492270469666
0.21342132985591888
0.3696369528770447
0.15429043769836426
0.08580858260393143
0.0864686518907547
0.056105609983205795
0.11221121996641159
0.21913142502307892
0.1683168262243271
0.08580858260393143
0.5544553995132446
0.0773514

KeyboardInterrupt: 